# Starting model here

In [ ]:
require 'torch'
require 'nn'
require 'rnn'
require 'csvigo'
require 'cutorch'
require 'cunn'
require 'cunnx'

--- Loading utility script
dofile("utils.lua")
dofile("model_utils.lua")

aurora_fn = '~/GitHub/DeepNLPQLearning/DO_NOT_UPLOAD_THIS_DATA/0-output/2012_aurora_shooting_first_sentence_numtext.csv'
nugget_fn = '~/GitHub/DeepNLPQLearning/DO_NOT_UPLOAD_THIS_DATA/0-output/aurora_nuggets_numtext.csv'
query_fn = '~/GitHub/DeepNLPQLearning/DO_NOT_UPLOAD_THIS_DATA/0-output/queries_numtext.csv'
auroras_fn = '~/GitHub/DeepNLPQLearning/DO_NOT_UPLOAD_THIS_DATA/0-output/2012_aurora_sentence_numtext.csv'

data_file = csvigo.load({path = aurora_fn, mode = "large"})
nugget_file = csvigo.load({path = nugget_fn, mode = "large"})
query_file =  csvigo.load({path = query_fn, mode = "large"})
sent_file =  csvigo.load({path = auroras_fn, mode = "large"})

In [ ]:
nstart = 2
nend = 40

In [ ]:
s_ss = geti_n(sent_file, nstart, nend)

In [ ]:
zero_or_x(1, sent_file[2])

In [ ]:
preds = torch.totable(torch.ones(#sent_file))

In [ ]:
tmp = {{1, 2 ,3 ,4}, {1, 3, 5, 4}}

In [ ]:
test = padZeros(tmp, 10)

In [ ]:
#test

In [ ]:
#test

In [ ]:
function getLastKTokens(x, K)
    local out = {}
    for i=0, K-1 do
        out[i+1] = x[#x-i]
    end 
    return out
end

function buildKSummary(preds, xs, K)
    local out = {}
    local out2 = {}
    for i=1, #xs do
        if i == 1 then 
            out[i] = zero_or_x(preds[i], {0} )
        else 
            --- Update it by adding xs_i and out_{i-1}
            out[i] = tableConcat( out[i-1], zero_or_x(preds[i], xs[i]) )
        end
    end
    local maxlen = 0
    for i=1, #out do
        out2[i] = getLastKTokens(out[i], K)
        maxlen = math.max(maxlen, #out2[i])
    end    
    return padZeros(out2, maxlen)
end

In [ ]:
pred_ss = buildKSummary(preds, sent_file, 50)

In [ ]:
sent_file[60]

In [ ]:
pred_ss[5]

In [ ]:
pred_ss[4]

In [ ]:
getLastKTokens(pred_ss[4], 3)

In [ ]:
#pred_ss

In [ ]:
pred_ss

In [ ]:
sent_file

In [ ]:
x = data_file
nstart = 2 
nend = 10

In [ ]:
--- This step is processing the data
x_ss  = geti_n(x, nstart, nend)
xout  = grabNsamples(x_ss, 1, #x_ss)     --- Extracting N samples
print('pass')

In [ ]:
mxl = 100

In [ ]:
xs  = padZeros(xout, mxl)                 --- Padding the data by the maximum length
qs2 = padZeros({qs}, 5)

In [ ]:
#xs

In [ ]:
qrep = repeatQuery(qs2[1], #xs)
preds = geti_n(preds_list, nstart, nend)
sumry_ss = buildSummary(preds, xs, 0)
print('pass')
sentences = LongTensor(xs):t()
summary = LongTensor(sumry_ss):t()
query = LongTensor(qrep):t()
actions = torch.Tensor(geti_n(action_list, nstart, nend)):resize(#xs, 1)
labels = torch.Tensor(geti_n(yrouge, nstart, nend))

In [ ]:
rK = 50
batch_size = 10
nepochs = 10
print_every = 1
embed_dim = 10
learning_rate = 0.1
usecuda = true

cuts = 4.                  --- This is the number of cuts we want
epsilon = 1.
base_explore_rate = 0.1
delta = 1./(nepochs/cuts) --- Only using epsilon greedy strategy for (nepochs/cuts)% of the epochs

torch.manualSeed(420)

function build_network(vocab_size, embed_dim)
    local model = nn.Sequential()
    :add(nn.LookupTableMaskZero(vocab_size, embed_dim)) -- will return a sequence-length x batch-size x embedDim tensor
    :add(nn.SplitTable(1, embed_dim)) -- splits into a sequence-length table with batch-size x embedDim entries
    :add(nn.Sequencer(nn.LSTM(embed_dim, embed_dim)))
    :add(nn.SelectTable(-1)) -- selects last state of the LSTM
    :add(nn.Linear(embed_dim, embed_dim)) -- map last state to a score for classification
    :add(nn.ReLU())
   return model
end

function build_model(vocab_size, embed_dim, outputSize, use_cuda)
    local mod1 = build_network(vocab_size, embed_dim)
    local mod2 = build_network(vocab_size, embed_dim)
    local mod3 = build_network(vocab_size, embed_dim)

    local mlp1 = nn.Sequential()
    mlp1:add(nn.Linear(1, embed_dim))
    mlp1:add(nn.ReLU())

    local ParallelModel = nn.ParallelTable()
    ParallelModel:add(mod1)
    ParallelModel:add(mod2)
    ParallelModel:add(mod3)
    ParallelModel:add(mlp1)

    local FinalMLP = nn.Sequential()
    FinalMLP:add(ParallelModel)
    FinalMLP:add(nn.JoinTable(2))
    FinalMLP:add( nn.Linear(embed_dim * 4, outputSize) )

    if use_cuda then
        return FinalMLP:cuda()
    else
        return FinalMLP
    end
end

vocab_sized = getVocabSize(data_file)                       --- getting length of dictionary
vocab_sizeq = getVocabSize(query_file)                      --- getting length of dictionary
vocab_size = math.max(vocab_sized, vocab_sizeq)

queries = grabNsamples(query_file, #query_file-1, nil)      --- Extracting all queries
nuggets = grabNsamples(nugget_file, #nugget_file-1, nil)    --- Extracting all samples
maxseqlend = getMaxseq(data_file)                             --- Extracting maximum sequence length
maxseqlenq = getMaxseq(query_file)                            --- Extracting maximum sequence length
maxseqlen = math.max(maxseqlenq, maxseqlend)

batchLSTM = build_model(vocab_size, embed_dim, 1, usecuda)
crit = nn.MSECriterion()

In [ ]:
batch_size = 10
nepochs = 1 
qs = queries[3]
x = data_file
model = batchLSTM
-- crit = crit
-- epsilon
-- delta
mxl = maxseqlen
-- base_explore_rate
-- print_every
-- nuggets
-- learning_rate
K = rK
use_cuda = usecuda

In [ ]:
if use_cuda then
  Tensor = torch.CudaTensor
  LongTensor = torch.CudaLongTensor
  crit = crit:cuda()
  print("Running DQN-LSTM with the GPU")
else
  Tensor = torch.Tensor
  LongTensor = torch.LongTensor
  print("Running DQN-LSTM with the CPU")
end

rscores, pscores, fscores = {}, {}, {}
yrouge = torch.totable(torch.randn(#x))
summary_list = populateOnes(#x, K)
action_list = torch.totable(torch.round(torch.rand(#x)))
preds_list = torch.totable(torch.round(torch.rand(#x)))
print("training model...")

In [ ]:
for epoch=0, nepochs, 1 do
    loss = 0                    --- Compute a new MSE loss each time
    --- Reset the rougue each time
    r_t1 , p_t1, f_t1 = 0., 0., 0.
    --- Looping over each bach of sentences for a given query
    nbatches = torch.floor( #x / batch_size)
    for minibatch = 1, nbatches do
        if minibatch == 1 then          -- Need +1 to skip the first row
            nstart = 2
            nend = torch.round(batch_size * minibatch)
        end
        if minibatch == nbatches then 
            nstart = nend + 1
            nend = #x
        end
        if minibatch > 1 and minibatch < nbatches then 
            nstart = nend + 1
            nend = torch.round(batch_size * minibatch)
        end
        --- This step is processing the data
        x_ss  = geti_n(x, nstart, nend)
        xout  = grabNsamples(x_ss, 1, #x_ss)     --- Extracting N samples
        print('pass')
        xs  = padZeros(xout, mxl)                 --- Padding the data by the maximum length
        qs2 = padZeros({qs}, 5)
        qrep = repeatQuery(qs2[1], #xs)
        preds = geti_n(preds_list, nstart, nend)
        sumry_ss = buildSummary(preds, xs, 0)

        sentences = LongTensor(xs):t()
        summary = LongTensor(sumry_ss):t()
        query = LongTensor(qrep):t()
        actions = torch.Tensor(geti_n(action_list, nstart, nend)):resize(#xs, 1)
        labels = torch.Tensor(geti_n(yrouge, nstart, nend))

        print(actions:sum(), actions:mean(), actions:min(), actions:max())

        if use_cuda then
             actions =  actions:cuda()
             labels = labels:cuda()
        end
        myPreds = model:forward({sentences, summary, query, actions})
        loss = loss + crit:forward(myPreds, labels)
        grads = crit:backward(myPreds, labels)
        model:backward({sentences, summary, query, actions}, grads)
        model:updateParameters(learning_rate)        -- Update parameters after each minibatch
        model:zeroGradParameters()

        if use_cuda then
            myPreds = myPreds:double()
        end

        preds = policy(myPreds, epsilon, #xs)
        --- Concatenating predictions into a summary
        predsummary = buildPredSummary(preds, xs, K)
        --- Initializing rouge metrics at time {t-1} and save scores
        for i=1, #predsummary do
            --- Calculating rouge scores; Call get_i_n() to cumulatively compute rouge
            rscores[i] = rougeRecall(geti_n(predsummary, 1, i), nuggets, K) - r_t1
            pscores[i] = rougePrecision(geti_n(predsummary, 1, i), nuggets, K) - p_t1
            fscores[i] = rougeF1(geti_n(predsummary, 1, i), nuggets, K) - f_t1
            r_t1, p_t1, f_t1 = rscores[i], pscores[i], fscores[i]
        end
        --- Updating change in rouge
        yrouge = updateTable(yrouge, fscores, nstart)
        preds_list = updateTable(preds_list, preds, nstart)
        action_list = updateTable(action_list, torch.totable(actions:double()), nstart)
        
        --- Calculating last one to see actual last rouge, without delta
        rscore = rougeRecall(predsummary, nuggets, K)
        pscore = rougePrecision(predsummary, nuggets, K)
        fscore = rougeF1(predsummary, nuggets, K)
        
        perf_string = string.format(
            "Epoch %i, sum(y)/len(y) = %i/%i, {Recall = %.6f, Precision = %.6f, F1 = %.6f}", 
            epoch, sumTable(preds_list), #preds_list, rscore, pscore, fscore
            )
        print(perf_string)
    end
    epsilon = epsilon - delta           --- Decreasing the epsilon greedy strategy
    if epsilon <= 0 then                --- leave a random exploration rate
        epsilon = base_explore_rate
    end
end

In [ ]:
predsummary[1][1]

In [ ]:
n2 = geti_n(nuggets, 2, #nuggets)

In [ ]:
unpackZeros(xs[1]), preds[1]

In [ ]:
summary[1]

In [ ]:
labels:size()

In [ ]:
predsummary[1]

In [ ]:
rougeF1(predsummary[1], nuggets, K)

In [ ]:
rougeF1(geti_n(predsummary, 1, 7)[1], nuggets, K)

In [ ]:
#sumry_ss

In [ ]:
function buildPredSummary(pred_action, xs) 
    local predsummary = {}
    --- This looks stupid but it's right because we have to retain
    --- the tmp1 when it's not 1, so it's a running total
    local tmp1 = {}
    for i=1, #xs do
        tmp = unpackZeros(xs[i])
        if pred_action[i]== 1 then
            predsummary[i] = tmp1
        else
            predsummary[i] = tmp1
        end
    end
    return predsummary
end

In [ ]:
pxs = buildPredSummary(preds, xs)

In [ ]:
unpackZeros(xs[1])

In [ ]:
x_ss

In [ ]:
rougeF1(geti_n(sumry_ss, 1, 2), nuggets, K)

In [ ]:
rougePrecision({unpackZeros(xs[1]), unpackZeros(xs[2]), unpackZeros(xs[3])}, nuggets, 3)

In [ ]:
preds[1], unpackZeros(xs[1])

In [ ]:
preds[1]

In [ ]:
out = {}
out[1] = zero_or_x(preds[1], unpackZeros(xs[1]))

In [ ]:
out[2] =  zero_or_x(preds[2], unpackZeros(xs[2]))
out[3] =  zero_or_x(preds[3], unpackZeros(xs[3]))

In [ ]:
tmp_preds  = {preds[1], preds[2], preds[3] }
tmp_xs = {unpackZeros(xs[1]), unpackZeros(xs[2]), unpackZeros(xs[3])} 

In [ ]:
tmp_xs[1]

In [ ]:
function buildPredSummary(preds, xs)
    local out = {}
    for i=1, #xs do
        if i == 1 then 
            out[i] = zero_or_x(preds[i], unpackZeros(xs[i]))
        else 
            --- Update it by adding xs_i and out_{i-1}
            out[i] =  zero_or_x(preds[i], unpackZeros(xs[i]))
        end
    end
    return out
end

In [ ]:
tst = buildPredSummary(geti_n(tmp_preds, 1, 2), tmp_xs)

In [ ]:
rougeF1(tst, nuggets)

In [ ]:
for i=1, 9 do
    print(rougePrecision(buildPredSummary(geti_n(preds, 1, i), geti_n(xout, 1, i)), nuggets))
end